# Kaggle Submission

This is a notebook that just formats data for a kaggle submission with the results of previous notebooks.

In [13]:
import pickle
import pandas as pd
import numpy as np
import patsy
import re

Previous results are loaded to be used later.

In [2]:
fileObj = open('./pickles/formula.pkl', 'rb')
formula = pickle.load(fileObj)
fileObj.close()

In [3]:
fileObj = open('./pickles/model.pkl', 'rb')
model = pickle.load(fileObj)
fileObj.close()

In [4]:
fileObj = open('./pickles/scaler.pkl', 'rb')
scaler = pickle.load(fileObj)
fileObj.close()

In [26]:
fileObj = open('./pickles/columns.pkl', 'rb')
columns = pickle.load(fileObj)
fileObj.close()

The formula is first replaced so that the target is removed.

In [10]:
formula = formula[12:]

In [19]:
test = pd.read_csv('./data/test.csv', index_col=0)

In [20]:
test.columns = test.columns.map(lambda x: x.replace(' ', ''))\
                    .map(lambda x: re.sub('^2', 'Seco', x))\
                    .map(lambda x: re.sub('^1', 'Fir', x))\
                    .map(lambda x: re.sub('^3', 'Third', x))\
                    .map(lambda x: x.replace('/', ''))

The formatting procedure from the previous two steps is used, though it is only important that columns that were previously used are selected and the same model is imported.

In [21]:
test = test.drop('PID', axis=1).fillna(0)

In [22]:
test.MSSubClass = test.MSSubClass.astype('category')
test.YrSold = test.YrSold.astype('category')

In [23]:
x1 = patsy.dmatrix(formula, test)

There are different dummy columns created due to the difference in test and train csv's.  Because it was not dealt with before, we now set the columns between the two datasets equivalent.  The information exclusive to the test set will be lost, unfortunately.

In [24]:
test_cols = set(x1.design_info.column_names)

In [27]:
orig_cols = set(columns)

In [28]:
diff_cols1 = orig_cols.difference(test_cols)
diff_cols2 = test_cols.difference(orig_cols)

In [29]:
x1 = pd.DataFrame(x1, columns=x1.design_info.column_names)

In [30]:
for column in diff_cols1:
    x1[column] = 0
    
for column in diff_cols2:
    x1.drop(column, axis=1, inplace=True)

In [32]:
x1 = x1[columns]
x1 = scaler.transform(x1)

In [34]:
y1 = pd.DataFrame(model.predict(x1), index=test.index, columns=['SalePrice'])

In [35]:
y1.to_csv('./scores/predictions.csv')